In [1]:
import numpy as np
import scipy as sc
from scipy import sparse
import time
import random

In [2]:
%load_ext autoreload
%autoreload 2
import oracles

In [3]:
class GDClassifier:
    """
    Реализация метода градиентного спуска для произвольного
    оракула, соответствующего спецификации оракулов из модуля oracles.py
    """

    def __init__(self, loss_function, step_alpha=1, step_beta=0, 
                 tolerance=1e-5, max_iter=1000, **kwargs):
        """
        loss_function - строка, отвечающая за функцию потерь классификатора. 
        Может принимать значения:
        - 'binary_logistic' - бинарная логистическая регрессия

        step_alpha - float, параметр выбора шага из текста задания

        step_beta- float, параметр выбора шага из текста задания

        tolerance - точность, по достижении которой, необходимо прекратить оптимизацию.
        Необходимо использовать критерий выхода по модулю разности соседних значений функции:
        если |f(x_{k+1}) - f(x_{k})| < tolerance: то выход 

        max_iter - максимальное число итераций     

        **kwargs - аргументы, необходимые для инициализации   
        """
        if loss_function == 'binary_logistic':
            self.oracle = oracles.BinaryLogistic(l2_coef=kwargs['l2_coef'])
        self.alpha = step_alpha
        self.beta = step_beta
        self.tolerance = tolerance
        self.max_iter = max_iter
        pass

    def fit(self, X, y, w_0=None, trace=False):
        """
        Обучение метода по выборке X с ответами y

        X - scipy.sparse.csr_matrix или двумерный numpy.array

        y - одномерный numpy array

        w_0 - начальное приближение в методе

        trace - переменная типа bool

        Если trace = True, то метод должен вернуть словарь history, содержащий информацию 
        о поведении метода. Длина словаря history = количество итераций + 1 (начальное приближение)

        history['time']: list of floats, содержит интервалы времени между двумя итерациями метода
        history['func']: list of floats, содержит значения функции на каждой итерации
        (0 для самой первой точки)
        """
        self.w = w_0.copy()
        w_k = w_0.copy()
        history = {'time': [0], 'func': [self.oracle.func(X, y, w_0)]}
        start = time.time()
        for k in range(1, self.max_iter):
            w_k1 = w_k - self.alpha / (k ** self.beta) * self.oracle.grad(X, y,
                                                                          w_k)

            if trace:
                history['func'].append(self.oracle.func(X, y, w_k1))
                history['time'].append(time.time() - start)
                start = time.time()
            if np.absolute(self.oracle.func(X, y, w_k1) -
                           self.oracle.func(X, y, w_k)) < self.tolerance:
                break
            w_k = w_k1.copy()
        self.w = w_k.copy()
        if trace:
            return history
        pass

    def predict(self, X):
        """
        Получение меток ответов на выборке X

        X - scipy.sparse.csr_matrix или двумерный numpy.array

        return: одномерный numpy array с предсказаниями
        """
        if sc.sparse.issparse(X):
            return np.sign(np.dot(self.w, X.todense().T))
        else:
            return np.sign(np.dot(self.w, X.T))
        pass

    def predict_proba(self, X):
        """
        Получение вероятностей принадлежности X к классу k

        X - scipy.sparse.csr_matrix или двумерный numpy.array

        return: двумерной numpy array, [i, k] значение соответветствует вероятности
        принадлежности i-го объекта к классу k 
        """
        if sc.sparse.issparse(X):
            return np.vstack((np.log(1 + np.exp(np.dot(self.w,
                                                       X.todense().T))),
                              np.log(1 + np.exp(-np.dot(self.w,
                                                        X.todense().T)))))
        else:
            return np.vstack((np.log(1 + np.exp(np.dot(self.w, X.T))),
                              np.log(1 + np.exp(-np.dot(self.w, X.T)))))
        pass

    def get_objective(self, X, y):
        """
        Получение значения целевой функции на выборке X с ответами y

        X - scipy.sparse.csr_matrix или двумерный numpy.array
        y - одномерный numpy array

        return: float
        """
        return self.oracle.func(X, y, self.w)
        pass

    def get_gradient(self, X, y):
        """
        Получение значения градиента функции на выборке X с ответами y

        X - scipy.sparse.csr_matrix или двумерный numpy.array
        y - одномерный numpy array

        return: numpy array, размерность зависит от задачи
        """
        return self.oracle.grad(X, y, self.w)
        pass

    def get_weights(self):
        """
        Получение значения весов функционала
        """ 
        return self.w
        pass


class SGDClassifier(GDClassifier):
    """
    Реализация метода стохастического градиентного спуска для произвольного
    оракула, соответствующего спецификации оракулов из модуля oracles.py
    """

    def __init__(self, loss_function, batch_size, step_alpha=1, step_beta=0, 
                 tolerance=1e-5, max_iter=1000, random_seed=153, **kwargs):
        """
        loss_function - строка, отвечающая за функцию потерь классификатора. 
        Может принимать значения:
        - 'binary_logistic' - бинарная логистическая регрессия

        batch_size - размер подвыборки, по которой считается градиент

        step_alpha - float, параметр выбора шага из текста задания

        step_beta- float, параметр выбора шага из текста задания

        tolerance - точность, по достижении которой, необходимо прекратить оптимизацию
        Необходимо использовать критерий выхода по модулю разности соседних значений функции:
        если |f(x_{k+1}) - f(x_{k})| < tolerance: то выход 


        max_iter - максимальное число итераций (эпох)

        random_seed - в начале метода fit необходимо вызвать np.random.seed(random_seed).
        Этот параметр нужен для воспроизводимости результатов на разных машинах.

        **kwargs - аргументы, необходимые для инициализации
        """
        super().__init__(loss_function=loss_function, step_alpha=step_alpha,
                         step_beta=step_beta, tolerance=tolerance,
                         max_iter=max_iter, l2_coef=kwargs['l2_coef'])
        self.batch_size = batch_size
        self.random_seed = random_seed
        pass

    def fit(self, X, y, w_0=None, trace=False, log_freq=1):
        """
        Обучение метода по выборке X с ответами y

        X - scipy.sparse.csr_matrix или двумерный numpy.array

        y - одномерный numpy array

        w_0 - начальное приближение в методе

        Если trace = True, то метод должен вернуть словарь history, содержащий информацию 
        о поведении метода. Если обновлять history после каждой итерации, метод перестанет 
        превосходить в скорости метод GD. Поэтому, необходимо обновлять историю метода лишь
        после некоторого числа обработанных объектов в зависимости от приближённого номера эпохи.
        Приближённый номер эпохи:
            {количество объектов, обработанных методом SGD} / {количество объектов в выборке}

        log_freq - float от 0 до 1, параметр, отвечающий за частоту обновления. 
        Обновление должно проиходить каждый раз, когда разница между двумя значениями приближённого номера эпохи
        будет превосходить log_freq.

        history['epoch_num']: list of floats, в каждом элементе списка будет записан приближённый номер эпохи:
        history['time']: list of floats, содержит интервалы времени между двумя соседними замерами
        history['func']: list of floats, содержит значения функции после текущего приближённого номера эпохи
        history['weights_diff']: list of floats, содержит квадрат нормы разности векторов весов с соседних замеров
        (0 для самой первой точки)
        """
        if sc.sparse.issparse(X):
            X = np.array(X.todense())

        random.seed(self.random_seed)

        ind = list(range(X.shape[0]))
        random.shuffle(ind)
        batches = [ind[i:i + self.batch_size] for i in range(0, len(ind),
                   self.batch_size)]

        self.w = w_0
        w_k = w_0
        history = {'epoch_num': [0], 'time': [0],
                   'func': [self.oracle.func(X, y, w_0)], 'weights_diff': [0]}
        old_epoch = 0
        start = time.time()

        for i in range(len(batches)):
            batch = batches[i]
            X_b = np.array([X[j] for j in batch])
            y_b = np.array([y[j] for j in batch])
            epoch_num = self.batch_size * i / X.shape[0]
            for k in range(self.max_iter):
                w_k1 = w_k - self.alpha / (k ** self.beta) *\
                    self.oracle.grad(X_b, y_b, w_k)

                if round(epoch_num - old_epoch, 8) > log_freq:
                    old_epoch = epoch_num
                    if trace:
                        history['func'].append(self.oracle.func(X_b, y_b,
                                                                w_k1))
                        history['epoch_num'].append(epoch_num)
                        history['weights_diff'].append(np.linalg.norm(
                                                       w_k1 - w_k) ** 2)
                        history['time'].append(time.time() - start)
                        start = time.time()
                if np.absolute(self.oracle.func(X_b, y_b, w_k1) -
                               self.oracle.func(X_b, y_b, w_k)) <\
                   self.tolerance:
                    break
                w_k = w_k1.copy()
            self.w = w_k
        if trace:
            return history
        pass


In [48]:
np.random.seed(10)
clf = GDClassifier(loss_function='binary_logistic', step_alpha=1,
    step_beta=0, tolerance=1e-4, max_iter=5, l2_coef=0.1)
l, d = 1000, 10
X = np.random.random((l, d))
y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
history = clf.fit(X, y, w_0=np.zeros(d), trace=True)
print(' '.join([str(x) for x in history['func']]))
clf.get_weights()

0.6931471805599454 0.6926868443805101 0.6925235796993467 0.6924122120106758 0.6923275050752085


array([ 0.00760802, -0.00994394, -0.02209522,  0.01089251, -0.01579711,
       -0.00945244,  0.00085363, -0.00370373, -0.02396217, -0.02186186])

In [49]:
np.random.seed(10)
clf = GDClassifier(loss_function='binary_logistic', step_alpha=1,
    step_beta=0, tolerance=1e-4, max_iter=5, l2_coef=0.1)
l, d = 1000, 10
X = np.random.random((l, d))
X = sparse.csr_matrix(X)
y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
history = clf.fit(X, y, w_0=np.zeros(d), trace=True)
print(' '.join([str(x) for x in history['func']]))
clf.get_weights()

0.6931471805599454 0.6931471805599454


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [47]:
np.random.seed(10)
clf = SGDClassifier(loss_function='binary_logistic', step_alpha=1,
    step_beta=0, tolerance=1e-4, max_iter=5, l2_coef=0.1, batch_size=50)
l, d = 1000, 10
X = np.random.random((l, d))
y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
history = clf.fit(X, y, w_0=np.zeros(d), trace=True, log_freq=0.299999)
# print(' '.join([str(x) for x in history['func']]))
print(history['epoch_num'])
print(history['time'])
clf.get_weights()

[0, 0.3, 0.6, 0.9]
[0, 0.009053945541381836, 0.010951519012451172, 0.01104116439819336]


array([ 0.03978721, -0.18664254, -0.13793378, -0.02476182, -0.15549331,
        0.01586502, -0.21661481, -0.2480776 , -0.11059034, -0.14013816])

In [192]:
clf = GDClassifier(loss_function='binary_logistic', step_alpha=1,
    step_beta=0, tolerance=1e-4, max_iter=5, l2_coef=0.1)

In [193]:
clf.fit(X, y, w_0=np.zeros(3), trace=True)
print(' '.join([str(x) for x in history['func']]))

0.6931471805599454 0.6926868443805101 0.6925235796993467 0.6924122120106758 0.6923275050752085


In [194]:
X_test = np.array([[-23, -5, -1]])
clf.predict_proba(X_test)

array([[2.54463117e-13],
       [2.90000000e+01]])

In [195]:
clf.predict(X_test)

array([-1.])

In [200]:
clf.get_objective(X, y)

0.6924122120106758

In [201]:
clf.get_gradient(X, y)

array([0.17758071, 0.20719767, 0.2195477 , 0.25327266, 0.27405911,
       0.18292316, 0.18354796, 0.27731672, 0.18651178, 0.18408566])

In [202]:
clf.get_weights()

array([ 0.00760802, -0.00994394, -0.02209522,  0.01089251, -0.01579711,
       -0.00945244,  0.00085363, -0.00370373, -0.02396217, -0.02186186])

In [207]:
clf = SGDClassifier(loss_function='binary_logistic', batch_size=5, step_alpha=1,
    step_beta=0, tolerance=1e-4, max_iter=5, l2_coef=0.1)

5 5


In [70]:

def BatchGenerator(sequence, batch_size):
    offset = 0
    while offset < len(sequence):
        ret_batch = sequence[offset: batch_size + offset]
        offset += batch_size
        yield ret_batch

In [59]:
# ind = range(X.shape[0])
ind = list(range(len(seq)))
random.shuffle(ind)
batch_size = 3
# bg = BatchGenerator(ind, batch_size)

In [56]:
list(zip(*[iter(ind)] * batch_size))

[(5, 0, 7), (2, 4, 9), (1, 3, 8)]

In [58]:
seq = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
# print(seq[bg])

In [62]:
batches = [ind[i:i + batch_size] for i in range(0, len(ind), batch_size)]

In [64]:
for batch in batches:
    print([seq[j] for j in batch])

['i', 'e', 'b']
['d', 'h', 'c']
['f', 'a', 'g']
['j']


In [19]:
l2_coef = 0.1
grad = - 1 / y.size * np.sum(np.exp(-y * np.dot(w, X.T)).reshape(-1, 1) * y.reshape(-1, 1) * X /
    np.array(1 + np.exp(-y * np.dot(w, X.T))).reshape(-1, 1),axis=0) + l2_coef * w
grad

ValueError: operands could not be broadcast together with shapes (1000,) (5,) 

In [17]:
X = X[:5]

In [18]:
X

array([[0.77132064, 0.02075195, 0.63364823, 0.74880388, 0.49850701,
        0.22479665, 0.19806286, 0.76053071, 0.16911084, 0.08833981],
       [0.68535982, 0.95339335, 0.00394827, 0.51219226, 0.81262096,
        0.61252607, 0.72175532, 0.29187607, 0.91777412, 0.71457578],
       [0.54254437, 0.14217005, 0.37334076, 0.67413362, 0.44183317,
        0.43401399, 0.61776698, 0.51313824, 0.65039718, 0.60103895],
       [0.8052232 , 0.52164715, 0.90864888, 0.31923609, 0.09045935,
        0.30070006, 0.11398436, 0.82868133, 0.04689632, 0.62628715],
       [0.54758616, 0.819287  , 0.19894754, 0.8568503 , 0.35165264,
        0.75464769, 0.29596171, 0.88393648, 0.32551164, 0.1650159 ]])

In [7]:
%%time
np.random.seed(17)
clf = GDClassifier(loss_function='binary_logistic', step_alpha=1,
    step_beta=0, tolerance=1e-4, max_iter=5, l2_coef=0.1)
l, d = 1000, 10
X = np.random.random((l, d))
y = np.random.randint(0, 2, l) * 2 - 1
w = np.random.random(d)
history = clf.fit(X, y, w_0=np.zeros(d), trace=True)
print(' '.join([str(x) for x in history['func']]))

0.6931471805599454 0.6922962722644881 0.6921673401771434 0.6921043596671055
Wall time: 28 ms
